In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install face_recognition

In [ ]:
# import required libraries
import json
import glob
import numpy as np
import cv2
import copy
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm

In [ ]:
#To get the average frame count 

#change the path accordingly
video_files =  glob.glob('/content/drive/MyDrive/Deepfake_videos/Deepfake_videos/*.mp4')
print(video_files)
#video_files =  glob.glob('/content/drive/MyDrive/Deepfake_videos/Deepfake_videos/*.mp4')
#video_files = video_files[1000:1200]
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frames per video:',np.mean(frame_count))

In [ ]:
!mkdir '/content/drive/MyDrive/Colab Notebooks/Preprocess_dfdc'

In [ ]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path) 
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image

In [ ]:
# process the frames
def create_face_videos(path_list,out_dir):

  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):

    out_path = os.path.join(out_dir,path.split('/')[-1])
    print("out_path = ",out_path)

    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue

    frames = []
    flag = 0
    face_all = []
    frames1 = []

    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 60, (500,500))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):

        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(500,500)))
            except: 
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [ ]:
create_face_videos(video_files,'/content/drive/MyDrive/Colab Notebooks/Preprocess_dfdc')